# Setting and suporting functions

In [1]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
import time
import json
from typing import List, Dict, Optional
from yt_dlp import YoutubeDL
# Load environment variables
load_dotenv()

# Get YouTube API key
API_KEY = os.getenv("H_YOUTUBE_API_KEY")
if not API_KEY:
    raise ValueError("API key not found. Please ensure the '.env' file is set up correctly.")
youtube = build("youtube", "v3", developerKey=API_KEY)

# Constants
MAX_RESULTS_PER_PAGE = 50
CACHE_DIR = "cache"
os.makedirs(CACHE_DIR, exist_ok=True)

In [2]:
# Supporting Functions

def get_video_stats(video_id: str) -> Optional[Dict]:
    """
    Fetch statistics for a given video ID.
    """
    try:
        response = youtube.videos().list(
            part="snippet,statistics,contentDetails,liveStreamingDetails,paidProductPlacementDetails,status,topicDetails",
            id=video_id
        ).execute()
    except Exception as e:
        print(f"Error fetching video data: {e}")
        return None

    if not response.get("items"):
        print(f"No video found with ID: {video_id}")
        return None

    video_data = response["items"][0]
    return {
        "title": video_data.get("snippet", {}).get("title", ""),
        "description": video_data.get("snippet", {}).get("description", ""),
        "channelTitle": video_data.get("snippet", {}).get("channelTitle", ""),
        "publishTime": video_data.get("snippet", {}).get("publishedAt", ""),
        "channelId": video_data.get("snippet", {}).get("channelId", ""),
        "thumbnail": video_data.get("snippet", {}).get("thumbnails", {}).get("standard", {}).get("url", ""),
        "tags": video_data.get("snippet", {}).get("tags", []),
        "topicCategories": video_data.get("topicDetails", {}).get("topicCategories", []),
        "categoryId": video_data.get("snippet", {}).get("categoryId", None),
        "liveBroadcastContent": video_data.get("snippet", {}).get("liveBroadcastContent", None),
        "duration": video_data.get("contentDetails", {}).get("duration", None),
        "has_caption": video_data.get("contentDetails", {}).get("caption", None),
        "licensedContent": video_data.get("contentDetails", {}).get("licensedContent", None),
        "definition": video_data.get("contentDetails", {}).get("definition", None),
        "contentRating": video_data.get("contentDetails", {}).get("contentRating", None),
        "madeForKids": video_data.get("status", {}).get("madeForKids", None),
        "privacyStatus": video_data.get("status", {}).get("privacyStatus", None),
        "viewCount": video_data.get("statistics", {}).get("viewCount", None),
        "likeCount": video_data.get("statistics", {}).get("likeCount", None),
        "commentCount": video_data.get("statistics", {}).get("commentCount", None),
        "dislikeCount": video_data.get("statistics", {}).get("dislikeCount", None),
        "favoriteCount": video_data.get("statistics", {}).get("favoriteCount", None),
        "paidProductPlacement": video_data.get("paidProductPlacementDetails", {}).get("hasPaidProductPlacement", None)
    }

def get_channel_stats(channel_id: str) -> Optional[Dict]:
    """
    Fetch statistics for a given channel ID.
    """
    try:
        response = youtube.channels().list(
            part="snippet,statistics,contentDetails,topicDetails",
            id=channel_id
        ).execute()
    except Exception as e:
        print(f"Error fetching channel data: {e}")
        return None

    if not response.get("items"):
        print(f"No channel found with ID: {channel_id}")
        return None

    channel_data = response["items"][0]
    return {
        "channel_title": channel_data["snippet"]["title"],
        "channel_description": channel_data["snippet"]["description"],
        "channel_custom_url": channel_data["snippet"]["customUrl"],
        "channel_thumbnail": channel_data["snippet"]["thumbnails"]["default"]["url"],
        "channel_published_at": channel_data["snippet"]["publishedAt"],
        "channel_view_count": channel_data["statistics"].get("viewCount", None),
        "channel_subscriber_count": channel_data["statistics"].get("subscriberCount", None),
        "channel_video_count": channel_data["statistics"].get("videoCount", None),
        "channel_playlist_count": channel_data["statistics"].get("playlistCount", None),
        "channel_comment_count": channel_data["statistics"].get("commentCount", None),
        "channel_live_count": channel_data["statistics"].get("liveCount", None),
        "channel_topic_categories": channel_data["topicDetails"].get("topicCategories", []),
        "channel_topic_ids": channel_data["topicDetails"].get("topicIds", []),
        "channel_country": channel_data["snippet"].get("country", None),
    }

def get_comments(video_id: str, cmt_count: int = 200, max_results: int = 100) -> Dict:
    """
    Fetch comments for a given video ID.
    """
    all_comments = []
    next_page_token = None
    search_pages = (cmt_count // max_results) + (1 if cmt_count % max_results else 0)

    for i in range(search_pages):
        try:
            comment_request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                pageToken=next_page_token,
                textFormat="plainText",
                maxResults=max_results,
                order="relevance"
            )
            comment_response = comment_request.execute()

            for item in comment_response['items']:
                top_comment = item['snippet']['topLevelComment']['snippet']
                all_comments.append({
                    'Timestamp': top_comment['publishedAt'],
                    'Username': top_comment['authorDisplayName'],
                    'VideoID': video_id,
                    'Comment': top_comment['textDisplay'],
                    'Date': top_comment.get('updatedAt', top_comment['publishedAt']),
                    "likeCount": top_comment["likeCount"],
                    "totalReplyCount": item["snippet"]["totalReplyCount"],
                })
            print(f"Page {i + 1} of comments fetched for video {video_id}")
            next_page_token = comment_response.get('nextPageToken')
            if not next_page_token:
                break
        except Exception as e:
            print(f"Error fetching comments for video {video_id}: {e}")
            break

    return {"top_comments": all_comments}

def get_transcript(video_id: str) -> Dict:
    """
    Fetch transcript for a given video ID.
    """
    try:
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
        transcript = transcript_list.find_transcript(['en'])
        transcript_src = transcript.fetch()
        transcript_txt = " ".join([line['text'] for line in transcript_src])
        return {"transcript": transcript_txt, "transcript_is_generated": transcript.is_generated}
    except Exception as e:
        print(f"Error fetching transcript for video {video_id}: {e}")
        return {"transcript": None, "transcript_is_generated": None}

def fetch_video_page(youtube, keyword: str, order: str, region: str, language: str, video_duration: str, 
                     published_start: str, published_end: str, video_quality: str, token: Optional[str] = None) -> Dict:
    """
    Fetch a single page of video search results from the YouTube API.
    """
    try:
        request = youtube.search().list(
            part="snippet",
            maxResults=MAX_RESULTS_PER_PAGE,
            q=keyword,
            order=order,
            type="video",
            regionCode=region,
            relevanceLanguage=language,
            videoDuration=video_duration,
            publishedAfter=published_start,
            publishedBefore=published_end,
            pageToken=token,
            videoDefinition=video_quality
        )
        return request.execute()
    except Exception as e:
        print(f"Error fetching video page: {e}")
        return {}

def extract_video_data(item: Dict, product_type: str, product_name: str) -> Dict:
    """
    Extract relevant video data from a single API response item.
    """
    return {
        "product_type": product_type,
        "product_name": product_name,
        "video_id": item["id"]["videoId"],
        "video_url": f"https://www.youtube.com/watch?v={item['id']['videoId']}",
        "video_title": item["snippet"]["title"],
        "video_description": item["snippet"]["description"],
        "video_channel": item["snippet"]["channelTitle"],
        "video_channel_id": item["snippet"]["channelId"],
        "video_published": item["snippet"]["publishedAt"],
        "video_thumbnail": item["snippet"]["thumbnails"]["default"]["url"]
    }

def save_to_cache(cache_file: str, keyword: str, response_list: List[Dict]) -> None:
    """
    Save the API response to a cache file.
    """
    cache_json = {'keyword': keyword, 'response_list': response_list}
    with open(cache_file, 'w') as f:
        json.dump(cache_json, f, indent=4, ensure_ascii=False)

def load_cache(cache_file: str) -> Dict:
    """
    Load cache from a file.
    """
    if os.path.exists(cache_file):
        with open(cache_file, "r") as f:
            return json.load(f)
    return {}

def update_cache(video_stats_cache: Dict, video_id: str, video_stats: Dict, cache_file: str) -> None:
    """
    Update cache with new video stats.
    """
    video_stats_cache[video_id] = video_stats
    with open(cache_file, "w") as f:
        json.dump(video_stats_cache, f)

def search_videos_by_keyword(youtube, product_type: str, product_name: str, region: str = 'US', language: str = "en", 
                            video_quality: str = "any", video_duration: str = "any", videos_count: int = 100, 
                            order: str = "viewCount", published_start: str = '2023-01-01T00:00:00Z', 
                            published_end: str = '2024-12-31T23:59:59Z', token: Optional[str] = None) -> List[Dict]:
    """
    Search for videos by keyword and return a list of video data.
    """
    keyword = f"{product_type} review"
    cache_file = os.path.join(CACHE_DIR, f'ytb_search_cache_{product_type}.json')
    # if cache_file exist load from cache file instead:
    response_list = load_cache(cache_file)
    if not response_list:
        response_list = []
    
        pages = (videos_count // MAX_RESULTS_PER_PAGE) + (1 if videos_count % MAX_RESULTS_PER_PAGE else 0)

        for i in range(pages):
            response = fetch_video_page(youtube, keyword, order, region, language, video_duration, 
                                    published_start, published_end, video_quality, token)
            if not response:
                break

            response_list.append(response)
            print(f"Page {i+1}: {response['pageInfo']['totalResults']} videos found. Next page token: {response.get('nextPageToken')}")

            token = response.get("nextPageToken")
            if not token:
                break
            # time.sleep(1)  # Avoid hitting API rate limits

        save_to_cache(cache_file, keyword, response_list)

    ytb_search_data = []
    for response in response_list:
        for item in response.get("items", []):
            video_data = extract_video_data(item, product_type, product_name)
            ytb_search_data.append(video_data)

    return ytb_search_data

def process_video_results(all_search_results: List[Dict], cache_file: str) -> List[Dict]:
    """
    Process video results, fetch stats, and update cache.
    """
    video_stats_cache = load_cache(cache_file)
    testing_search = []

    for result in all_search_results:
        video_id = result["video_id"]
        if video_id in video_stats_cache:
            print(f"Using cached data for video ID: {video_id}")
            video_stats = video_stats_cache[video_id]
        else:
            video_stats = get_video_stats(video_id)
            if video_stats:
                update_cache(video_stats_cache, video_id, video_stats, cache_file)
                print(f"Fetched and cached data for video ID: {video_id}")
            else:
                print(f"Failed to fetch data for video ID: {video_id}")
                continue

        result.update(video_stats)
        testing_search.append(result)

    return testing_search

def fetch_and_process_videos(product_type: str, product_name: str = "", videos_count: int = 1000) -> pd.DataFrame:
    """
    Fetch videos, process stats, fetch transcripts, and save results to a CSV file.
    """
    # Define cache file paths
    product_type_file = product_type.replace(" ", "_").lower()
    cache_file = os.path.join(CACHE_DIR, f"video_stats_cache_{product_type_file}.json")
    output_file = os.path.join(CACHE_DIR, f"ytb_search_combined_results_{product_type_file}.csv")

    # Load cache if it exists, or initialize an empty cache
    if os.path.exists(cache_file):
        with open(cache_file, "r") as f:
            video_stats_cache = json.load(f)
    else:
        video_stats_cache = {}

    # Fetch videos by keyword
    print(f"Searching for videos related to '{product_type}'...")
    search_results = search_videos_by_keyword(
        youtube=youtube,
        product_type=product_type,
        product_name=product_name,
        videos_count=videos_count,
        order="relevance",
        region="US",
        language="en",
        video_duration="medium",
        published_start='2023-01-01T00:00:00Z',
        published_end='2024-12-31T23:59:59Z'
    )
    print(f"Found {len(search_results)} videos for {product_type}.")

    # Process each video
    all_search_results = []
    for result in search_results:
        video_id = result["video_id"]

        # Fetch video stats (from cache or API)
        if video_id in video_stats_cache:
            print(f"Using cached data for video ID: {video_id}")
            video_stats = video_stats_cache[video_id]
        else:
            video_stats = get_video_stats(video_id)
            if video_stats:
                # Fetch transcript
                transcript_data = get_transcript(video_id)
                video_stats.update(transcript_data)  # Add transcript to video stats

                # Update cache
                video_stats_cache[video_id] = video_stats
                with open(cache_file, "w") as f:
                    json.dump(video_stats_cache, f)
                print(f"Fetched and cached data for video ID: {video_id}")
            else:
                print(f"Failed to fetch data for video ID: {video_id}")
                continue  # Skip to the next result if API call fails

        # Update result with video stats and add to the list
        result.update(video_stats)
        all_search_results.append(result)

    print(f"Total videos processed: {len(all_search_results)}")

    # Convert to DataFrame and save
    ytb_search_df = pd.DataFrame(all_search_results)
    ytb_search_df.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")

    return ytb_search_df



def save_results_to_csv(testing_search: List[Dict], keyword: str) -> None:
    """
    Save results to a CSV file.
    """
    keyword_file = keyword.replace(" ", "_").lower()
    ytb_search_df = pd.DataFrame(testing_search)
    output_file = os.path.join(CACHE_DIR, f"ytb_search_results_{keyword_file}.csv")
    ytb_search_df.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")


def fetch_video_metadata(video_id: str, ydl: YoutubeDL, cache_file: str, video_info_cache: Dict) -> Optional[Dict]:
    """
    Fetch metadata for a video using yt-dlp and update the cache.
    """
    if video_id in video_info_cache:
        print(f"Skipping {video_id}: Already in cache.")
        return video_info_cache[video_id]

    url = f"https://www.youtube.com/watch?v={video_id}"
    try:
        print(f"Fetching info for {video_id}...")
        info = ydl.extract_info(url, download=False)
        video_info_cache[video_id] = info
        return info
    except Exception as e:
        print(f"Error fetching info for {video_id}: {e}")
        return None

In [18]:
if __name__ == "__main__":
    product_type_to_search = "Galaxy ZFlip 6"
    df = fetch_and_process_videos(product_type_to_search, videos_count=10) #small number for testing
    print(df.head())

Using cached search results from cache\ytb_search_cache_galaxy_zflip_6.json
Using cached video stats for video ID: _wDMdqbhyrg
Using cached video stats for video ID: XNyrZ8FMfMY
Using cached video stats for video ID: 5t1okmz8Pyw
Using cached video stats for video ID: ulXeKHtePDI
Using cached video stats for video ID: 2GaS97uC77w
Using cached video stats for video ID: oBP25Rvy_OI
Using cached video stats for video ID: BsL-kXX3Ns8
Using cached video stats for video ID: A763kAaz1SE
Using cached video stats for video ID: iDxyFtsWkUY
Using cached video stats for video ID: zijUjJdegI8
Using cached video stats for video ID: MCxxpMXFTg4
Using cached video stats for video ID: IdAl5O0YqAU
Using cached video stats for video ID: DLzKXBAAEcU
Using cached video stats for video ID: xQ0hJS41MdQ
Using cached video stats for video ID: 9SVPHHFtScE
Using cached video stats for video ID: B4bbF_lo9mk
Using cached video stats for video ID: KYDsc5UEANU
Using cached video stats for video ID: -dMTU6deTPA
Using 

In [3]:
search_products = [
    "COSRX Snail Mucin Serum",
    "Tesla Model Y",
    "Samsung OLED TV",
    "Liquid Death",
    "Stanley tumbler",
    "Apple AirTag",
    "Meta quest 3",
    "Samsung Galaxy Watch 7",
    "Macbook Pro M4",
    "Lululemon Align Pants",
]

In [ ]:
for keyword in search_products[7:]:
    ytb_search_df = fetch_and_process_videos(product_type=keyword, videos_count=500)
    print(ytb_search_df.head())
    print(f"Finished processing {keyword} videos. Total videos: {len(ytb_search_df)}")
    

Fetching search results from YouTube API for keyword: Samsung Galaxy Watch 7 review
Page 1: 925858 videos found. Next page token: CDIQAA
Page 2: 925858 videos found. Next page token: CGQQAA
Page 3: 926810 videos found. Next page token: CJYBEAA
Page 4: 925857 videos found. Next page token: CMgBEAA
Page 5: 925857 videos found. Next page token: CPoBEAA
Page 6: 925857 videos found. Next page token: CKwCEAA
Page 7: 925857 videos found. Next page token: CN4CEAA
Page 8: 925859 videos found. Next page token: CJADEAA
Page 9: 925857 videos found. Next page token: CMIDEAA
Page 10: 925857 videos found. Next page token: CPQDEAA
Error fetching transcript for video GhZq1wwC82U: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=GhZq1wwC82U! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/

In [5]:
import logging 

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
def fetch_video_metadata(video_id: str, ydl: YoutubeDL, video_info_cache: Dict) -> Optional[Dict]:
    """
    Fetch metadata for a video using yt-dlp and update the cache.
    """
    if video_info_cache.get(video_id):
        print(f"Skipping {video_id}: Already in cache.")
        return None  # Return None consistently

    url = f"https://www.youtube.com/watch?v={video_id}"
    try:
        print(f"Fetching info for {video_id}...")
        info = ydl.extract_info(url, download=False)
        print(f"Successfully fetched info for {video_id}.")
        return info
        
    except Exception as e:
        print(f"Error fetching info for {video_id}: {e}")
        return None

def fetch_metadata_for_dataframe(df: pd.DataFrame, category: str, delay: float = 1.0) -> pd.DataFrame:
    """
    Fetch metadata for all videos in a DataFrame and update the cache.

    Args:
        df: The DataFrame containing video IDs.
        category: A string representing the category (used for cache file naming).
        delay: The delay in seconds between yt-dlp calls (for rate limiting).

    Returns:
        The updated DataFrame with a new 'metadata' column.
    """
    metadata_cache_file = os.path.join(CACHE_DIR, f"video_info_cache_{category}.json")
    video_info_cache = load_cache(metadata_cache_file)

    ydl_opts = {"quiet": True}
    ydl = YoutubeDL(ydl_opts)

    metadata_dict = {}
    for idx, video_id in enumerate(df["video_id"]):
        try:
            metadata = fetch_video_metadata(video_id, ydl, video_info_cache)
            if metadata:
                metadata_dict[video_id] = metadata
        except Exception as e:
            logging.error(f"Error processing video ID {video_id}: {e}")  # Log the error

        # Update cache
        if len(metadata_dict) == 10:
            # update video_info_cache every 10 videos
            video_info_cache.update(metadata_dict)
            with open(metadata_cache_file, "w") as f:
                json.dump(video_info_cache, f)
            logging.info(f"Processed {idx + 1} videos.  Cache updated.")
            # Reset metadata_list
            metadata_dict = {}

        #time.sleep(delay)  # Enforce delay

    # Update cache
    video_info_cache.update(metadata_dict)
    with open(metadata_cache_file, "w") as f:
        json.dump(video_info_cache, f)  
    # update_cache(video_info_cache, video_id, metadata, metadata_cache_file) #update cache
    logging.info("Metadata fetching complete. Cache updated.")

    df["metadata"] = metadata_dict
    return df

In [ ]:


# Example usage
if __name__ == "__main__":
    product_type = "lululemon align pants"
    ytb_search_df = fetch_and_process_videos(product_type=product_type, videos_count=500)

    # Fetch metadata as a post-processing step
    ytb_search_df = fetch_metadata_for_dataframe(ytb_search_df, category="fashion_50")

    # Save the updated DataFrame
    output_file = os.path.join(CACHE_DIR, f"ytb_search_results_with_metadata_{product_type.replace(' ', '_').lower()}.csv")
    ytb_search_df.to_csv(output_file, index=False)
    print(f"Data with metadata saved to {output_file}")

# Clean text dataset 

In [33]:
import pandas as pd

def add_cols(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add new columns to the DataFrame and clean existing ones.
    """
    # Convert duration to seconds (e.g., PT4M20S to 260)
    try:
        df["duration_seconds"] = df["duration"].apply(lambda x: pd.to_timedelta(x).seconds)
    except Exception as e:
        print(f"Error converting duration to seconds: {e}")
        df["duration_seconds"] = None  # Fallback to None if conversion fails

    # Convert 'video_published' to datetime format
    df["video_published"] = pd.to_datetime(df["video_published"], errors="coerce")

    # Check for invalid dates
    if df["video_published"].isnull().any():
        print("Warning: Some values in 'video_published' could not be converted to datetime!")

    # Extract 'publish_month' in 'YYYY-MM' format
    df["publish_month"] = df["video_published"].dt.strftime("%Y-%m")

    # Convert numeric columns to numeric type
    numeric_cols = ["viewCount", "likeCount", "commentCount", "duration_seconds"]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Sort the DataFrame by 'viewCount' in descending order
    df_sorted = df.sort_values(by=["viewCount"], ascending=False)
    return df_sorted

def drop_cols_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drop unnecessary columns and rows, and clean the DataFrame.
    """
    # Drop columns with all null values
    df = df.dropna(axis=1, how="all")

    # Drop rows with all null values
    df = df.dropna(how="all")

    # Drop duplicates, reset index, and sort by 'viewCount'
    df = df.drop_duplicates().reset_index(drop=True).sort_values(by=["viewCount"], ascending=False)
    # drop unnecessary columns
    try:
        df = df.drop(['liveBroadcastContent', 'contentRating', 'video_channel', 'title', 'video_channel_id', 'privacyStatus'], axis=1)
    except KeyError:
        pass

    # drop rows with null values in 'transcript'
    # df = df.dropna(subset=["transcript"])

    
    # drop dumplicate columns
    df = df.loc[:,~df.columns.duplicated()]
    return df

In [4]:
import glob

csv_combined = glob.glob(os.path.join(CACHE_DIR, "ytb_search_combined_results_*.csv"))
csv_combined = [file for file in csv_combined if not file.endswith('_metadata.csv')]
csv_combined

['cache\\ytb_search_combined_results_apple_airtag.csv',
 'cache\\ytb_search_combined_results_cosrx_snail_mucin_serum.csv',
 'cache\\ytb_search_combined_results_liquid_death.csv',
 'cache\\ytb_search_combined_results_lululemon_align_pants.csv',
 'cache\\ytb_search_combined_results_macbook_pro_m4.csv',
 'cache\\ytb_search_combined_results_meta_quest_3.csv',
 'cache\\ytb_search_combined_results_samsung_galaxy_watch_7.csv',
 'cache\\ytb_search_combined_results_samsung_oled_tv.csv',
 'cache\\ytb_search_combined_results_stanley_tumbler.csv',
 'cache\\ytb_search_combined_results_tesla_model_y.csv']

In [35]:
# clean the data and save it
for csv_search_file in csv_combined:
    df = pd.read_csv(csv_search_file)
    df = add_cols(df)
    df = drop_cols_rows(df)
    name_f = "cleaned_" + os.path.basename(csv_search_file)
    df.to_csv(os.path.join(CACHE_DIR, name_f), index=False)
    print(f"Data cleaned and saved to {os.path.join(CACHE_DIR, name_f)}")
    # remove timezone information
    df["video_published"] = df["video_published"].dt.tz_localize(None)
    # save to excel
    df.to_excel(os.path.join(CACHE_DIR, name_f.replace(".csv", ".xlsx")), index=False)
    print(f"Data cleaned and saved to {os.path.join(CACHE_DIR, name_f.replace('.csv', '.xlsx'))}")

Data cleaned and saved to cache\cleaned_ytb_search_combined_results_apple_airtag.csv
Data cleaned and saved to cache\cleaned_ytb_search_combined_results_apple_airtag.xlsx
Data cleaned and saved to cache\cleaned_ytb_search_combined_results_cosrx_snail_mucin_serum.csv
Data cleaned and saved to cache\cleaned_ytb_search_combined_results_cosrx_snail_mucin_serum.xlsx
Data cleaned and saved to cache\cleaned_ytb_search_combined_results_galaxy_zflip_6.csv
Data cleaned and saved to cache\cleaned_ytb_search_combined_results_galaxy_zflip_6.xlsx
Data cleaned and saved to cache\cleaned_ytb_search_combined_results_liquid_death.csv
Data cleaned and saved to cache\cleaned_ytb_search_combined_results_liquid_death.xlsx
Data cleaned and saved to cache\cleaned_ytb_search_combined_results_lululemon_align_pants.csv
Data cleaned and saved to cache\cleaned_ytb_search_combined_results_lululemon_align_pants.xlsx
Data cleaned and saved to cache\cleaned_ytb_search_combined_results_macbook_pro_m4.csv
Data cleaned a

In [5]:
import glob

csv_combined = glob.glob(os.path.join(CACHE_DIR, "cleaned_ytb_search_combined_*.csv"))
#csv_combined = [file for file in csv_combined if not file.endswith('_metadata.csv')]
csv_combined

['cache\\cleaned_ytb_search_combined_results_apple_airtag.csv',
 'cache\\cleaned_ytb_search_combined_results_cosrx_snail_mucin_serum.csv',
 'cache\\cleaned_ytb_search_combined_results_liquid_death.csv',
 'cache\\cleaned_ytb_search_combined_results_lululemon_align_pants.csv',
 'cache\\cleaned_ytb_search_combined_results_macbook_pro_m4.csv',
 'cache\\cleaned_ytb_search_combined_results_meta_quest_3.csv',
 'cache\\cleaned_ytb_search_combined_results_samsung_galaxy_watch_7.csv',
 'cache\\cleaned_ytb_search_combined_results_samsung_oled_tv.csv',
 'cache\\cleaned_ytb_search_combined_results_stanley_tumbler.csv',
 'cache\\cleaned_ytb_search_combined_results_tesla_model_y.csv']

In [6]:
search_products.sort()
search_products

['Apple AirTag',
 'COSRX Snail Mucin Serum',
 'Liquid Death',
 'Lululemon Align Pants',
 'Macbook Pro M4',
 'Meta quest 3',
 'Samsung Galaxy Watch 7',
 'Samsung OLED TV',
 'Stanley tumbler',
 'Tesla Model Y']

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def process_csv_with_fuzzy(csv_file, search_term="review", threshold=58):
    """
    Process CSV file to add fuzzy matching columns
    
    Args:
        csv_file (str): Path to CSV file
        search_term (str): Term to match against titles
        threshold (int): Fuzzy matching threshold
    
    Returns:
        pd.DataFrame: Processed dataframe with new columns
    """
    df = pd.read_csv(csv_file)
    
    # Calculate fuzzy ratios for all titles
    df['fuzzy_ratio_title'] = df['video_title'].apply(
        lambda x: fuzz.partial_ratio(search_term.lower(), str(x).lower())
    )
    
    # Add binary matched column
    df['matched_title'] = (df['fuzzy_ratio_title'] > threshold).astype(int)

    # Calculate fuzzy ratios for all descriptions
    df['fuzzy_ratio_description'] = df['video_description'].apply(
        lambda x: fuzz.partial_ratio(search_term.lower(), str(x).lower())
    )

    # Add binary matched column
    df['matched_description'] = (df['fuzzy_ratio_description'] > threshold).astype(int)

    # sort by 'viewCount' in descending order for matched_title=1
    df = df.sort_values(by=["matched_description", "matched_title", "viewCount"], ascending=[False, False, False])
    
    return df

# Process each CSV file
processed_dfs = []
for i, csv in enumerate(csv_combined):
    df_processed = process_csv_with_fuzzy(csv, search_term=search_products[i])
    
    # Print matching titles and their ratios
    matching_titles = df_processed[df_processed['matched_title'] == 1]
    print(f"\nMatching titles in {csv}:")
    print(matching_titles[['video_title', 'fuzzy_ratio_title']].to_string())

    # Print matching descriptions and their ratios
    matching_descriptions = df_processed[df_processed['matched_description'] == 1]
    print(f"\nMatching descriptions in {csv}:")
    print(matching_descriptions[['video_description', 'fuzzy_ratio_description']].to_string())


    
    processed_dfs.append(df_processed)
    
    # Print summary
    print(f"\nTotal matches Title for {csv}: {len(matching_titles)}")
    print(f"Total matches Description for {csv}: {len(matching_descriptions)}")
len(processed_dfs)


Matching titles in cache\cleaned_ytb_search_combined_results_apple_airtag.csv:
                                                                                                      video_title  fuzzy_ratio_title
12                                               TRACK YOUR CAR with Apple AirTags! The BEST Anti-Theft Solution!                100
16                           Apple Watch Series 9 Tips, Tricks &amp; Hidden Features | YOU ABSOLUTELY MUST KNOW!!                 67
19                                                Versteckte Funktion: Wusstest du, dass dein AirTag das kann?! 😲                 67
23                                                                   Apple Watch Series 9 Review - 6 Months Later                 67
25                               Apple Watch Ultra 2 vs Apple Watch Series 10: Every single difference explained!                 67
29                                                   Apple Intelligence Setup: Your iPhone Will Never Be The Same         

10

In [ ]:
for i, df in enumerate(processed_dfs):
    output_path = csv_combined[i].replace('.csv', '_matched_keywords.csv')
    df.to_csv(output_path, index=False)
    print(f"Processed data saved to {output_path}")


Processed data saved to cache\cleaned_ytb_search_combined_results_apple_airtag_matched_keywords.csv
Processed data saved to cache\cleaned_ytb_search_combined_results_cosrx_snail_mucin_serum_matched_keywords.csv
Processed data saved to cache\cleaned_ytb_search_combined_results_liquid_death_matched_keywords.csv
Processed data saved to cache\cleaned_ytb_search_combined_results_lululemon_align_pants_matched_keywords.csv
Processed data saved to cache\cleaned_ytb_search_combined_results_macbook_pro_m4_matched_keywords.csv
Processed data saved to cache\cleaned_ytb_search_combined_results_meta_quest_3_matched_keywords.csv
Processed data saved to cache\cleaned_ytb_search_combined_results_samsung_galaxy_watch_7_matched_keywords.csv
Processed data saved to cache\cleaned_ytb_search_combined_results_samsung_oled_tv_matched_keywords.csv
Processed data saved to cache\cleaned_ytb_search_combined_results_stanley_tumbler_matched_keywords.csv
Processed data saved to cache\cleaned_ytb_search_combined_resu

In [6]:
import glob

csv_matched = glob.glob(os.path.join(CACHE_DIR, "cleaned_ytb_search_combined_*matched_keywords.csv"))
#csv_combined = [file for file in csv_combined if not file.endswith('_metadata.csv')]
csv_matched

['cache\\cleaned_ytb_search_combined_results_apple_airtag_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_cosrx_snail_mucin_serum_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_liquid_death_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_lululemon_align_pants_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_macbook_pro_m4_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_meta_quest_3_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_samsung_galaxy_watch_7_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_samsung_oled_tv_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_stanley_tumbler_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_tesla_model_y_matched_keywords.csv']

In [ ]:
for csv in csv_matched:
    df = pd.read_csv(csv)
    # Use parentheses and & or | operators for boolean operations in pandas
    df_matched = df[
        (df['matched_title'] == 1) | 
        (df['matched_description'] == 1)
    ].copy()  # Use .copy() to avoid SettingWithCopyWarning
    
    print(f"\nAnalysis for {csv}:")
    print(f"Total matched rows: {len(df_matched)}")
    
    # Count non-null transcripts in matched rows
    transcript_count = df_matched['transcript'].notna().sum()
    print(f"Total non-empty transcripts: {transcript_count}")


    # try to recover transcript for videos with no transcript using get_transcript
    for index, row in df_matched.iterrows():
        if pd.isnull(row['transcript']):
            video_id = row['video_id']
            transcript_dict = get_transcript(video_id)
            df_matched.at[index, 'transcript'] = transcript_dict['transcript']
            df_matched.at[index, 'transcript_is_generated'] = transcript_dict['transcript_is_generated']
            print(f"Transcript recovered for video ID: {video_id}")

    # # Count non-null transcripts in matched rows
    # transcript_count = df_matched['transcript'].notna().sum()
    # print(f"Total non-empty transcripts after recovery: {transcript_count}")

    # # Save updated DataFrame with recovered transcripts
    # df_matched.to_csv(csv, index=False)
    # print(f"Processed data saved to {csv}")


Analysis for cache\cleaned_ytb_search_combined_results_apple_airtag_matched_keywords.csv:
Total matched rows: 306
Total non-empty transcripts: 221

Analysis for cache\cleaned_ytb_search_combined_results_cosrx_snail_mucin_serum_matched_keywords.csv:
Total matched rows: 288
Total non-empty transcripts: 155

Analysis for cache\cleaned_ytb_search_combined_results_liquid_death_matched_keywords.csv:
Total matched rows: 307
Total non-empty transcripts: 285

Analysis for cache\cleaned_ytb_search_combined_results_lululemon_align_pants_matched_keywords.csv:
Total matched rows: 286
Total non-empty transcripts: 282

Analysis for cache\cleaned_ytb_search_combined_results_macbook_pro_m4_matched_keywords.csv:
Total matched rows: 276
Total non-empty transcripts: 192

Analysis for cache\cleaned_ytb_search_combined_results_meta_quest_3_matched_keywords.csv:
Total matched rows: 449
Total non-empty transcripts: 421

Analysis for cache\cleaned_ytb_search_combined_results_samsung_galaxy_watch_7_matched_key

In [ ]:
# append metadata to the dataframes
import glob
import pandas as pd

csv_matched = glob.glob(os.path.join(CACHE_DIR, "cleaned_ytb_search_combined_*matched_keywords.csv"))
for csv in csv_matched:
    df = pd.read_csv(csv)
    # Use parentheses and & or | operators for boolean operations in pandas
    df_matched = df[
        (df['matched_title'] == 1) | 
        (df['matched_description'] == 1)    
    ]

    df_matched = fetch_metadata_for_dataframe(df_matched, category="fashion_50")
    df_matched.to_csv(csv, index=False)

In [7]:
import glob

csv_matched = glob.glob(os.path.join(CACHE_DIR, "cleaned_ytb_search_combined_*matched_keywords.csv"))
#csv_combined = [file for file in csv_combined if not file.endswith('_metadata.csv')]
csv_matched

['cache\\cleaned_ytb_search_combined_results_apple_airtag_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_cosrx_snail_mucin_serum_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_liquid_death_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_lululemon_align_pants_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_macbook_pro_m4_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_meta_quest_3_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_samsung_galaxy_watch_7_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_samsung_oled_tv_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_stanley_tumbler_matched_keywords.csv',
 'cache\\cleaned_ytb_search_combined_results_tesla_model_y_matched_keywords.csv']

In [9]:
# fetch meta data for each df, then save to csv and excel

for csv in csv_matched[:]:
    ytb_search_df = pd.read_csv(csv)
    category = os.path.basename(csv).split("results_")[-1].split(".")[0].replace("_matched_keywords", "")
    print(category)
    ytb_search_df = fetch_metadata_for_dataframe(ytb_search_df, category=category)
    # ytb_search_df.to_csv(os.path.join(CACHE_DIR, f"ytb_search_combined_results_{category}_metadata.csv"), index=False)
    # ytb_search_df.to_excel(os.path.join(CACHE_DIR, f"ytb_search_combined_results_{category}_metadata.xlsx"), index=False)
    print(f"Finished processing {category} videos. Total videos: {len(ytb_search_df)}")
    


apple_airtag
Skipping cB9ukX5_tp8: Already in cache.
Skipping CLOOLOdg5Jw: Already in cache.
Skipping cSkYaHGPfBQ: Already in cache.
Skipping wM16xWaJHbU: Already in cache.
Skipping zj-dM0B1l28: Already in cache.
Skipping F2tXw74fyxc: Already in cache.
Skipping R4JXjsbx45U: Already in cache.
Skipping hiivC_4li8Q: Already in cache.
Skipping 44FEs8jAN9Y: Already in cache.
Skipping xNAJkhDW8tg: Already in cache.
Skipping 7mOoMwxpN04: Already in cache.
Skipping BmgCyR-JXhA: Already in cache.
Skipping zhvv5_Kag28: Already in cache.
Skipping okD4MfKU6Y4: Already in cache.
Skipping lB1ZHmhmFHA: Already in cache.
Skipping TLVxLnh2Pak: Already in cache.
Skipping wROcuvDJscg: Already in cache.
Skipping X27UudoqbVg: Already in cache.
Skipping sfCUVTmeQ98: Already in cache.
Skipping 7LK9QcH-Dtg: Already in cache.
Skipping PUkDzWVo1Xc: Already in cache.
Skipping zutcGa73kbM: Already in cache.
Skipping lPSVhTGkorE: Already in cache.
Skipping zeL2iaS2qOk: Already in cache.
Skipping e2tMZFev0ow: Alrea

2025-02-14 16:27:53,806 - INFO - Metadata fetching complete. Cache updated.


Finished processing apple_airtag videos. Total videos: 306
cosrx_snail_mucin_serum
Skipping qZlItN__Cnw: Already in cache.
Skipping sRPVUxpMzIM: Already in cache.
Skipping ymM3KdVrChc: Already in cache.
Skipping PppMg_5Xnr8: Already in cache.
Skipping aALYla5Uqt4: Already in cache.
Skipping A1h8vOb0h6Q: Already in cache.
Skipping 8M1eKfOEkTQ: Already in cache.
Skipping mNj4heCfdsY: Already in cache.
Skipping A1IVGzMIRi0: Already in cache.
Skipping JtgqoAZ0rx0: Already in cache.
Skipping Lw2aJiLH_40: Already in cache.
Skipping kcF5z8JsuVk: Already in cache.
Skipping aKtn7A9VLA0: Already in cache.
Skipping 30Ypi9sfRR4: Already in cache.
Skipping 9uJMMuZAT_Y: Already in cache.
Skipping kfbJ4bulK0A: Already in cache.
Skipping yR5VdfTmxHI: Already in cache.
Skipping qs85rDxHp0c: Already in cache.
Skipping mm-35E-PpIM: Already in cache.
Skipping THVsvCSbIh0: Already in cache.
Skipping zu3gNnAcYzo: Already in cache.
Skipping DW9vON1DRA8: Already in cache.
Skipping zRNhuV-FiL4: Already in cach

2025-02-14 16:27:57,534 - INFO - Metadata fetching complete. Cache updated.


Finished processing cosrx_snail_mucin_serum videos. Total videos: 288
liquid_death
Skipping UfpBPk8HiaY: Already in cache.
Skipping ZKulKTBG4z8: Already in cache.
Skipping 2NVKMZEut_o: Already in cache.
Skipping W98dsnZsix4: Already in cache.
Skipping fe9H3nrUE64: Already in cache.
Skipping g4R5Nz5agz4: Already in cache.
Skipping TcejRPUhF3c: Already in cache.
Skipping -awPOByIu_0: Already in cache.
Skipping OhsS4d9b9qQ: Already in cache.
Skipping 5yXa6OmUPMA: Already in cache.
Skipping z8CI3hhFfVg: Already in cache.
Skipping _0XQs2ntvL4: Already in cache.
Fetching info for aMGs9HhjeWQ...


...

Finished processing samsung_oled_tv videos. Total videos: 288
stanley_tumbler
Skipping v8QJABGKn_k: Already in cache.
Skipping PgWQ2eJN2I8: Already in cache.
Skipping i_7Zfq4Atas: Already in cache.
Skipping CkqWVCLfPFo: Already in cache.
Skipping MZE7sn-Ujl8: Already in cache.
Skipping ka6A-Upjjd8: Already in cache.
Skipping smYfftZaTIM: Already in cache.
Skipping Fhp-08zcM64: Already in cache.
Skipping CYo1yEYwdD4: Already in cache.
Skipping qU-p9XSg1jo: Already in cache.
Skipping 0vvlDHyHvjY: Already in cache.
Skipping Oe-5EaG-UJ8: Already in cache.
Skipping c1oAdgz0Tbw: Already in cache.
Skipping lLwsns_q3O0: Already in cache.
Skipping QxqaXoVIUwE: Already in cache.
Skipping 0_WoerQCO1E: Already in cache.
Skipping MDVcDMtAuxs: Already in cache.
Skipping bXXKc0tZdn8: Already in cache.
Skipping RCA33LZwCno: Already in cache.
Skipping -OGooYyRP6A: Already in cache.
Skipping DZQlYHR1wsY: Already in cache.
Skipping dAwvKnWDv5s: Already in cache.
Skipping rVmGZuGqk3I: Already in cache.
Sk

2025-02-14 16:28:33,508 - INFO - Metadata fetching complete. Cache updated.


Finished processing stanley_tumbler videos. Total videos: 353
tesla_model_y
Skipping iueGI4CzP-0: Already in cache.
Skipping vwR0pjwuIAQ: Already in cache.
Skipping gy6SkVi9rvI: Already in cache.
Skipping mmjN3Z4HcEI: Already in cache.
Skipping 1En6pmcGJdk: Already in cache.
Skipping gTbGSmBC6DE: Already in cache.
Skipping zAdV29_IXWc: Already in cache.
Skipping BNZIcSJw-t8: Already in cache.
Skipping DvLnhRi6iVM: Already in cache.
Skipping Ms5j8A0inwY: Already in cache.
Skipping G8JSwNwXu9A: Already in cache.
Skipping 9WN9o4HK8CQ: Already in cache.
Skipping wMY_93dTQ_w: Already in cache.
Skipping 2pPhzzCsBaM: Already in cache.
Skipping E0417QP2bV0: Already in cache.
Skipping p5pI8UZZfes: Already in cache.
Skipping W_lY1xtopYM: Already in cache.
Skipping 7GmiRMXkBxo: Already in cache.
Skipping gCg7j7vYlY0: Already in cache.
Skipping juFIozsUelo: Already in cache.
Skipping f_H_xjCy9VU: Already in cache.
Skipping xp5kD9IPt44: Already in cache.
Skipping sRa5YEyzeUE: Already in cache.
Skip

2025-02-14 16:28:38,411 - INFO - Metadata fetching complete. Cache updated.


Finished processing tesla_model_y videos. Total videos: 466


In [43]:
cache_dir="cache"
json_files = glob.glob(os.path.join(cache_dir, "video_info_cache_*.json"))
json_files

['cache\\video_info_cache_apple_airtag.json',
 'cache\\video_info_cache_cosrx_snail_mucin_serum.json',
 'cache\\video_info_cache_liquid_death.json',
 'cache\\video_info_cache_lululemon_align_pants.json',
 'cache\\video_info_cache_macbook_pro_m4.json',
 'cache\\video_info_cache_meta_quest_3.json',
 'cache\\video_info_cache_samsung_galaxy_watch_7.json',
 'cache\\video_info_cache_samsung_oled_tv.json',
 'cache\\video_info_cache_stanley_tumbler.json',
 'cache\\video_info_cache_tesla_model_y.json']

In [11]:
def process_video_metadata(df, cache_data):
    """
    Process video metadata from cached JSON files and merge with existing DataFrame
    
    Args:
        df (pd.DataFrame): Input DataFrame containing video_ids
        cache_dir (str): Directory containing cached JSON files
        
    Returns:
        pd.DataFrame: DataFrame with merged metadata
    """
    import json
    import os
    
    result_list = []
    
    # Load cached video info
    
    if not cache_data:
        print("No video info found in cache.")
        # return None
    
    # Process each video
    for video_id in df.video_id:
        video_data = cache_data.get(video_id, {})
        if not video_data:
            print(f"No video info found for video ID: {video_id}")
            continue
        append_dict = {"video_id": video_id}
        
        # Extract formats data
        # Extract formats data - Fixed this part
        for fmt in video_data.get("formats", []):  # Changed from cache_data to video_data
            if fmt.get("format_id") == "sb0":
                append_dict["merged_screenshot"] = fmt.get("fragments")
        
        # Extract other metadata
        append_dict.update({
            "video_chapters": video_data.get("chapters"),
            "heatmap": video_data.get("heatmap"),
            "automatic_captions": video_data.get("automatic_captions", {}).get("en", []),
            # "duration": video_data.get("duration"),
            # "view_count": video_data.get("view_count"),
            # "like_count": video_data.get("like_count"),
            # "comment_count": video_data.get("comment_count"),
            # "average_rating": video_data.get("average_rating")
        })
        # print(append_dict)
        
        result_list.append(append_dict)
    
    # Convert to DataFrame and merge
    result_df = pd.DataFrame(result_list)
    # left join on video_id
    final_df = pd.merge(df, result_df, how="left", on="video_id")
    
    # remove duplicates
    final_df = final_df.drop_duplicates(subset=['video_id'])
    
    return final_df

# Usage example:
cache_dir="cache"
json_files = glob.glob(os.path.join(cache_dir, "video_info_cache_*.json"))

for i, csv_file in enumerate(csv_matched):
    df = pd.read_csv(csv_file)
    print(f"Processing {csv_file}...")
    
    # Process video metadata
    print(f"Processing {json_files[i]}...")
    video_info = load_cache(json_files[i])
    print(len(video_info))
    df_with_metadata = process_video_metadata(df, video_info)

    # Create a binary column indicating if both transcript and screenshots are available
    df_with_metadata['transcript_screenshot_not_null'] = (
        df_with_metadata['merged_screenshot'].notna() & 
        df_with_metadata['transcript'].notna()
    ).astype(int)
    
    # sort by transcript_screenshot_not_null then viewCount
    df_with_metadata = df_with_metadata.sort_values(by=["transcript_screenshot_not_null", "viewCount"], ascending=[False, False])
    # Save processed DataFrame
    output_file = csv_file.replace('.csv', '_with_metadata.csv')
    df_with_metadata.to_csv(output_file, index=False)

    # save to excel
    df_with_metadata.to_excel(output_file.replace('.csv', '.xlsx'), index=False)
    
    print(f"\nProcessed {csv_file}:")
    print(f"Original columns: {len(df.columns)}")
    print(f"New columns: {len(df_with_metadata.columns)}")
    print(f"Rows processed: {len(df_with_metadata)}")

Processing cache\cleaned_ytb_search_combined_results_apple_airtag_matched_keywords.csv...
Processing cache\video_info_cache_apple_airtag.json...
419

Processed cache\cleaned_ytb_search_combined_results_apple_airtag_matched_keywords.csv:
Original columns: 33
New columns: 38
Rows processed: 306
Processing cache\cleaned_ytb_search_combined_results_cosrx_snail_mucin_serum_matched_keywords.csv...
Processing cache\video_info_cache_cosrx_snail_mucin_serum.json...
419

Processed cache\cleaned_ytb_search_combined_results_cosrx_snail_mucin_serum_matched_keywords.csv:
Original columns: 33
New columns: 38
Rows processed: 288
Processing cache\cleaned_ytb_search_combined_results_liquid_death_matched_keywords.csv...
Processing cache\video_info_cache_liquid_death.json...
368
No video info found for video ID: aMGs9HhjeWQ
No video info found for video ID: hgiV-800TjU
No video info found for video ID: UDxq0vnGNhg
No video info found for video ID: htPlnC4Ket8
No video info found for video ID: PI8CDi71fXQ


In [12]:
cache_dir="cache"
final_csvs = glob.glob(os.path.join(cache_dir, "cleaned_ytb_search_combined_*_matched_keywords_with_metadata.csv"))
final_csvs

['cache\\cleaned_ytb_search_combined_results_apple_airtag_matched_keywords_with_metadata.csv',
 'cache\\cleaned_ytb_search_combined_results_cosrx_snail_mucin_serum_matched_keywords_with_metadata.csv',
 'cache\\cleaned_ytb_search_combined_results_liquid_death_matched_keywords_with_metadata.csv',
 'cache\\cleaned_ytb_search_combined_results_lululemon_align_pants_matched_keywords_with_metadata.csv',
 'cache\\cleaned_ytb_search_combined_results_macbook_pro_m4_matched_keywords_with_metadata.csv',
 'cache\\cleaned_ytb_search_combined_results_meta_quest_3_matched_keywords_with_metadata.csv',
 'cache\\cleaned_ytb_search_combined_results_samsung_galaxy_watch_7_matched_keywords_with_metadata.csv',
 'cache\\cleaned_ytb_search_combined_results_samsung_oled_tv_matched_keywords_with_metadata.csv',
 'cache\\cleaned_ytb_search_combined_results_stanley_tumbler_matched_keywords_with_metadata.csv',
 'cache\\cleaned_ytb_search_combined_results_tesla_model_y_matched_keywords_with_metadata.csv']

In [13]:
import json
import ast
import requests
import os
from io import BytesIO
from PIL import Image

def process_json(json_string):
    try:
        python_dict = ast.literal_eval(json_string) # Safely evaluates to Python dict
        json_string = json.dumps(python_dict) #Converts to proper JSON string
        data = json.loads(json_string)
        return data
    except (ValueError, SyntaxError, json.JSONDecodeError) as e:
        print(f"Error processing JSON: {e}")
        return None

def save_images_from_urls(image_dict, output_dir="images"):
    """
    Downloads images from a list of URLs and saves them locally.

    Args:
        image_urls: A list of image URLs.
        output_dir: The directory to save the images (created if it doesn't exist).
    """

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for video_id in image_dict:
        image_urls = [img['url'] for img in image_dict[video_id]]
        for i, url in enumerate(image_urls):
            try:
                response = requests.get(url, stream=True)
                response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

                image = Image.open(BytesIO(response.content))
                filename = os.path.join(output_dir, f"{video_id}_{i+1}.jpg")  #Safe filename creation
                image.save(filename)
                print(f"Image {i+1} saved to: {filename}")

            except requests.exceptions.RequestException as e:
                print(f"Error downloading image {i+1} from {url}: {e}")
            except IOError as e:
                print(f"Error saving image {i+1} from {url}: {e}")
            except Exception as e: #Catch any other unexpected error
                print(f"An unexpected error occurred for image {i+1} from {url}: {e}")


In [ ]:

# img_dict_tech = {}
# for idx, row in tech_df_100.iterrows():
#     if pd.notnull(row['merged_screenshot']):
#         img_dict_tech[row['video_id']] = process_json(row['merged_screenshot'])
# len(img_dict_tech), img_dict_tech.keys()

for csv in final_csvs:
    df = pd.read_csv(csv)
    print(f"Processing {csv}...")
    img_dict = {}
    for idx, row in df.iterrows():
        if pd.notnull(row['merged_screenshot']):
            img_dict[row['video_id']] = process_json(row['merged_screenshot'])
    print(len(img_dict), img_dict.keys())
    save_images_from_urls(img_dict, output_dir="cache/images")

House keeping

In [3]:
import os
import shutil

# Create the directories
os.makedirs('cache/csv_files', exist_ok=True)
os.makedirs('cache/json_files', exist_ok=True)
os.makedirs('cache/xlsx_files', exist_ok=True)

# Move files to their respective folders
for file in os.listdir('cache/'):
    if file.endswith('.csv'):
        shutil.move(f'cache/{file}', f'cache/csv_files/{file}')
    elif file.endswith('.json'):
        shutil.move(f'cache/{file}', f'cache/json_files/{file}')
    elif file.endswith('.xlsx'):
        shutil.move(f'cache/{file}', f'cache/xlsx_files/{file}')